The below command should produce a table of output, it just means that you do have a GPU instance, if you don't, go to "Runtime" and "Change Runtime Type" and make sure and use "GPU" hardware acceleration


In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
!ln -sf /content/gdrive/MyDrive/tribunal/gpt2/models/ models
!ln -sf /content/gdrive/MyDrive/tribunal/gpt2/checkpoint/ checkpoint 
!cd checkpoint ; ln -sf ../models/355M/ 355M
!ls -al checkpoint/355M/

In [ ]:
!pip3 install gpt-2-simple==0.7.1
!pip3 install tensorflow-gpu==1.15

The following lines of python run inference on the model, generating output based on the prompt (prefix). 'length' is the amount of tokens (words) that are produced, 'temperature' makes the text more diverse the higher it is (0.1 less diverse text, 1.0 very diverse text) 'nsamples' is the amount of 'length' sized texts produced, 'batch_size' is the amount of samples created as a time (i.e if you had 'nsamples' = 100 and 'batch_size=20', the response would be 5 chunks of 20 'length' sized texts) 'top_k' restricts the amount of words the model will choose from for the next predicted word, if set to 0 this behavior is turned off, 'prefix' is the text you prompt the model with and 'include_prefix' includes the prefix in the output.

In [ ]:
import gpt_2_simple as gpt2
import os
import requests

base_fname = "/content/gdrive/MyDrive/tribunal/gpt2/outputs/"
model_name = "run1"
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, model_name)

mylines = []
myOutput = []

for i in range(0,100):
  mylines = gpt2.generate(sess, run_name=model_name,
	  length=1024,
	  temperature=0.9, #larger number means more diverse text
  	nsamples=1, #you can generate multiple samples per run
	  batch_size=1, #batch size must divide equally into nsamples
	  top_k=0,
	  prefix="<|startoftext|>\n",
	  include_prefix=False,
    return_as_list=True,
    truncate='<|endoftext|>')
  if i % 25 == 0:
    # reset session
    gpt2.reset_session(sess)
    sess = gpt2.start_tf_sess()
    gpt2.load_gpt2(sess, model_name)
  fnum = i
  fname = base_fname + "%04d" % fnum + ".txt"
  fh = open(fname, "w", encoding='utf-8')
  for line in mylines:
    fh.write(line)
  fh.close()
  print("Wrote file:",fname)

gpt2.reset_session(sess)

